#  Import

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_train = pd.read_csv('../train.csv')

##  dummies

In [4]:
#df_train = pd.concat([df_train, pd.get_dummies(df_train['PdDistrict'], prefix='PdDistrict')], axis=1)
#df_train = pd.concat([df_train, pd.get_dummies(df_train['DayOfWeek'], prefix='DayOfWeek')], axis=1)
#df_train = pd.concat([df_train, pd.get_dummies(df_train.Dates.map(lambda x: pd.to_datetime(x).hour), prefix="hour")], axis=1)
#df_train = pd.concat([df_train, pd.get_dummies(df_train.Dates.map(lambda x: pd.to_datetime(x).month), prefix="month")], axis=1)
#df_train = pd.concat([df_train, pd.get_dummies(df_train.Dates.map(lambda x: pd.to_datetime(x).year), prefix="year")], axis=1)

# preprocessing

## drop non-use 

In [5]:
df_train = df_train.drop('Descript', axis = 1)
df_train = df_train.drop('Resolution', axis = 1)
df_train = df_train.drop('Address', axis = 1)
df_train = df_train.drop('DayOfWeek', axis = 1)
df_train = df_train.drop('PdDistrict', axis = 1)
df_train = df_train.drop('Dates', axis = 1)

# Start Feature addressing 

In [6]:
x_mid = df_train['X'].mean()

In [7]:
y_mid = df_train['Y'].mean()

In [8]:
df_train['new_X'] = df_train['X'] - x_mid
df_train['new_Y'] = df_train['Y'] - y_mid

In [11]:
df_train['R'] = np.sqrt(df_train['new_X']**2 + df_train['new_Y']**2)

In [12]:
df_train['Theta'] = np.arctan2(df_train['new_Y'], df_train['new_X'])

# Let's train 

In [14]:
df_train = df_train.drop('new_X',axis = 1)
df_train = df_train.drop('new_Y',axis = 1)

In [22]:
columns = df_train.columns.values[3:5]

In [23]:
columns

array(['R', 'Theta'], dtype=object)

In [24]:
x_data = df_train[columns].values
y_data = df_train['Category'].values

In [25]:
from sklearn.tree import DecisionTreeClassifier

In [26]:
model = DecisionTreeClassifier(max_depth=7)
model.fit(x_data, y_data)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

# test preprocessing 

In [27]:
test_df = pd.read_csv('../test.csv')

In [28]:
test_df = test_df.drop('Dates', axis = 1)
test_df = test_df.drop('DayOfWeek', axis = 1)
test_df = test_df.drop('PdDistrict', axis = 1)
test_df = test_df.drop('Address', axis = 1)
test_df = test_df.drop('Id', axis = 1)

#  do ML

In [29]:
test_df['new_X'] = test_df['X'] - x_mid
test_df['new_Y'] = test_df['Y'] - y_mid

In [30]:
test_df['R'] = np.sqrt(test_df['new_X']**2 + test_df['new_Y']**2)
test_df['Theta'] = np.arctan2(test_df['new_Y'], test_df['new_X'])

In [31]:
test_data = test_df[columns]

output = model.predict_proba(test_data)

b = model.classes_
columns2 = np.append(['Id'], b)
test_data = test_df.values
result = np.c_[test_data[:,0].astype(int), output.astype(float)]
df_result = pd.DataFrame(result[:,:],columns=columns2)

df_result['Id'] = df_result.index

In [32]:
df_result.to_csv('SecondTest.csv', index=False)